# Question 1  
**Explain the fundamental differences between DDL, DML, and DQL commands in SQL. Provide one example for each type of command.**

### Answer  
DDL (Data Definition Language) defines and manages the structure of a database. It includes operations such as creating, altering, or deleting tables. These commands modify schema-level objects and do not handle actual data. Examples include `CREATE`, `ALTER`, and `DROP`.  
DML (Data Manipulation Language) is responsible for modifying the data stored within tables. It enables inserting new records, updating existing ones, and deleting data. Examples include `INSERT`, `UPDATE`, and `DELETE`.  
DQL (Data Query Language) is used to retrieve data from databases. The main command in this category is `SELECT`, which allows users to extract and analyze information based on specific conditions.  
**Examples:**  
- DDL → `CREATE TABLE Students (...);`  
- DML → `INSERT INTO Students VALUES (...);`  
- DQL → `SELECT * FROM Students;`

# Question 2  
**What is the purpose of SQL constraints? Name and describe three common types of constraints, providing a simple scenario where each would be useful.**

### Answer  
SQL constraints enforce rules that maintain accuracy, reliability, and consistency of data in a database. They prevent invalid or inconsistent data from being entered.  
A PRIMARY KEY constraint uniquely identifies each record, ensuring no duplicates or NULL values—useful for a Students table where every student must have a unique ID.  
A FOREIGN KEY constraint ensures referential integrity by linking related tables. For example, an Orders table should reference valid CustomerIDs from a Customers table.  
A UNIQUE constraint ensures that all values in a column are different. This is essential for columns like email addresses during user registration, where duplicates cannot be allowed.  
Constraints thus protect data quality and prevent anomalies during inserts, updates, or deletes.

# Question 3  
**Explain the difference between LIMIT and OFFSET clauses in SQL. How would you use them together to retrieve the third page of results, assuming each page has 10 records?**

### Answer  
LIMIT and OFFSET are used to implement pagination in SQL queries. LIMIT specifies how many rows should be returned, while OFFSET tells the database how many rows to skip before starting to return results.  
For example, if a page contains 10 records, page 1 retrieves rows 0–9, page 2 retrieves 10–19, and page 3 retrieves 20–29.  
To fetch the third page, we skip the first 20 rows using OFFSET and retrieve the next 10 using LIMIT.  
**Example:**  
```sql
SELECT * FROM table_name  
LIMIT 10 OFFSET 20;
```

# Question 4  
**What is a Common Table Expression (CTE) in SQL, and what are its main benefits? Provide a simple SQL example demonstrating its usage.**

### Answer  
A Common Table Expression (CTE) is a temporary, named result set defined using the WITH clause. It improves code readability by organizing complex queries into clean, reusable sections. CTEs help eliminate nested subqueries, support recursion, and enhance maintainability.  
They exist only during the execution of a single query and do not consume memory permanently.  
**Example:**  
```sql
WITH HighPrice AS (
    SELECT ProductName, Price FROM Products WHERE Price > 100
)
SELECT * FROM HighPrice;
```

# Question 5  
**Describe the concept of SQL Normalization and its primary goals. Briefly explain the first three normal forms (1NF, 2NF, 3NF).**

### Answer  
SQL Normalization is a database design technique aimed at reducing redundancy, ensuring consistency, and preventing update anomalies.  
1NF requires atomic values—no repeating groups or multivalued attributes.  
2NF requires a table to be in 1NF and that all non-key columns depend on the whole primary key, eliminating partial dependencies in composite key tables.  
3NF eliminates transitive dependencies, ensuring non-key attributes depend only on the primary key.  
Normalization creates a clean, efficient, and scalable database structure.

In [9]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('ECommerceDB.db')
cursor = conn.cursor()

In [10]:

cursor.execute("DROP TABLE IF EXISTS Categories;")
cursor.execute("DROP TABLE IF EXISTS Products;")
cursor.execute("DROP TABLE IF EXISTS Customers;")
cursor.execute("DROP TABLE IF EXISTS Orders;")

cursor.execute("""CREATE TABLE Categories(
    CategoryID INT PRIMARY KEY,
    CategoryName TEXT UNIQUE NOT NULL
);""")

cursor.execute("""CREATE TABLE Products(
    ProductID INT PRIMARY KEY,
    ProductName TEXT UNIQUE NOT NULL,
    CategoryID INT,
    Price REAL NOT NULL,
    StockQuantity INT,
    FOREIGN KEY(CategoryID) REFERENCES Categories(CategoryID)
);""")

cursor.execute("""CREATE TABLE Customers(
    CustomerID INT PRIMARY KEY,
    CustomerName TEXT NOT NULL,
    Email TEXT UNIQUE,
    JoinDate TEXT
);""")

cursor.execute("""CREATE TABLE Orders(
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate TEXT NOT NULL,
    TotalAmount REAL,
    FOREIGN KEY(CustomerID) REFERENCES Customers(CustomerID)
);""")

conn.commit()


OperationalError: database is locked

In [ ]:

cursor.executemany("INSERT INTO Categories VALUES (?,?)", [
(1,'Electronics'),(2,'Books'),(3,'Home Goods'),(4,'Apparel')
])

cursor.executemany("INSERT INTO Products VALUES (?,?,?,?,?)", [
(101,'Laptop Pro',1,1200.00,50),
(102,'SQL Handbook',2,45.50,200),
(103,'Smart Speaker',1,99.99,150),
(104,'Coffee Maker',3,75.00,80),
(105,'Novel : The Great SQL',2,25.00,120),
(106,'Wireless Earbuds',1,150.00,100),
(107,'Blender X',3,120.00,60),
(108,'T-Shirt Casual',4,20.00,300)
])

cursor.executemany("INSERT INTO Customers VALUES (?,?,?,?)", [
(1,'Alice Wonderland','alice@example.com','2023-01-10'),
(2,'Bob the Builder','bob@example.com','2022-11-25'),
(3,'Charlie Chaplin','charlie@example.com','2023-03-01'),
(4,'Diana Prince','diana@example.com','2021-04-26')
])

cursor.executemany("INSERT INTO Orders VALUES (?,?,?,?)", [
(1001,1,'2023-04-26',1245.50),
(1002,2,'2023-10-12',99.99),
(1003,1,'2023-07-01',145.00),
(1004,3,'2023-01-14',150.00),
(1005,2,'2023-09-24',120.00),
(1006,1,'2023-06-19',20.00)
])

conn.commit()


In [ ]:

df = pd.read_sql_query("""SELECT c.CustomerName, c.Email, COUNT(o.OrderID) AS TotalOrders
FROM Customers c LEFT JOIN Orders o ON c.CustomerID=o.CustomerID
GROUP BY c.CustomerID ORDER BY c.CustomerName;""", conn)
df